In [79]:
# define some useful functions

def read_bytes(path):
    with open(path, 'rb') as f:
        return f.read()

def write_bytes(path, bytes):
    with open(path, 'wb') as f:
        f.write(bytes)

def print_list(llist):
    for i in llist:
        print(i)


In [80]:
# define png chunk parser

def parse_png_chunk_data_list(bytes):
    output_data_list = []
    i = 0
    if bytes[i:i+8] != b'\x89PNG\r\n\x1a\n':
        raise Exception('invalid PNG header')
    output_data_list.append({
        'type': 'header',
        'start': i,
        'end': i+8,
        'length': 8,
    })
    i += 8
    while True:
        if i >= len(bytes):
            raise Exception(f'unexpected end of file: i={i}, len(bytes)={len(bytes)}')
        start = i
        length = int.from_bytes(bytes[i:i+4], 'big')
        i += 4
        chunk_type = bytes[i:i+4]
        i += 4
        i += length
        i += 4 # CRC
        end = i
        output_data_list.append({
            'type': 'chunk',
            'chunk_type': chunk_type,
            'start': start,
            'end': end,
            'length': end - start,
        })
        if chunk_type == b'IEND':
            break
    return output_data_list


In [81]:
# sample file names

sample_charcard_face_0_fn   = 'HCCP_F_20230913224620491.png'
sample_charcard_face_100_fn = 'HCCP_F_20230913225210929.png'


In [82]:
# sample file bytes and parsed data

sample_charcard_face_0_bytes = read_bytes(sample_charcard_face_0_fn)
sample_charcard_face_0_chunk_data_list = parse_png_chunk_data_list(sample_charcard_face_0_bytes)
print_list(sample_charcard_face_0_chunk_data_list)


{'type': 'header', 'start': 0, 'end': 8, 'length': 8}
{'type': 'chunk', 'chunk_type': b'IHDR', 'start': 8, 'end': 33, 'length': 25}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 33, 'end': 8237, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 8237, 'end': 16441, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 16441, 'end': 24645, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 24645, 'end': 32849, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 32849, 'end': 41053, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 41053, 'end': 49257, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 49257, 'end': 57461, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 57461, 'end': 65665, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 65665, 'end': 73869, 'length': 8204}
{'type': 'chunk', 'chunk_type': b'IDAT', 'start': 73869, 'end': 82073, 'length': 8204}
{'t

In [83]:
# notice that there are extra data at the end of the png file

print(f'len(sample_charcard_face_0_bytes)= {len(sample_charcard_face_0_bytes)}')
print(f'sample_charcard_face_0_chunk_data_list[-1][\'end\']= {sample_charcard_face_0_chunk_data_list[-1]["end"]}')

if len(sample_charcard_face_0_bytes) > sample_charcard_face_0_chunk_data_list[-1]['end']:
    print('extra data at the end of the png file')
else:
    print('no extra data at the end of the png file')


len(sample_charcard_face_0_bytes)= 179279
sample_charcard_face_0_chunk_data_list[-1]['end']= 84674
extra data at the end of the png file


In [84]:
# get the extra bytes from the char card file
sample_charcard_face_0_extra_bytes = sample_charcard_face_0_bytes[sample_charcard_face_0_chunk_data_list[-1]['end']:]

# write the extra bytes to a file
write_bytes('sample_charcard_face_0_extra_bytes.bin', sample_charcard_face_0_extra_bytes)

In [85]:
offset0 = 0

# 0: header
assert(sample_charcard_face_0_extra_bytes[offset0:offset0+4] == b'\x64\x00\x00\x00')
offset0 += 4

# 1: data: contains char "HCChara"
print(f'data.1.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+1], 'little')
offset0 += 1
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
print(bytes.decode('utf-8'))

# 2: data: contains version?
print(f'data.2.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+1], 'little')
offset0 += 1
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
print(bytes.decode('utf-8'))

# 3: data: contains png data
print(f'data.3.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.3.png'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 4: data
print(f'data.4.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.4.bin'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

# 5: data
print(f'data.5.offset = {hex(offset0)}')
length = int.from_bytes(sample_charcard_face_0_extra_bytes[offset0:offset0+4], 'little')
length += 4 # bug?
offset0 += 4
bytes = sample_charcard_face_0_extra_bytes[offset0:offset0+length]
offset0 += length
out_fn = 'sample_charcard_face_0_extra_bytes.5.bin'
print(f'save to {out_fn}')
write_bytes(out_fn, bytes)

print(f'offset0 = {offset0}')
print(f'len(sample_charcard_face_0_extra_bytes) = {len(sample_charcard_face_0_extra_bytes)}')


data.1.offset = 0x4
【HCChara】
data.2.offset = 0x12
0.0.0
data.3.offset = 0x18
save to sample_charcard_face_0_extra_bytes.3.png
data.4.offset = 0x60dd
save to sample_charcard_face_0_extra_bytes.4.bin
data.5.offset = 0x6253
save to sample_charcard_face_0_extra_bytes.5.bin
offset0 = 94605
len(sample_charcard_face_0_extra_bytes) = 94605


In [86]:
# helper function to try parsing sample_charcard_face_0_extra_bytes.4.bin

def read_a_bytes(bytes, offset):
    length = int.from_bytes(bytes[offset:offset+1], 'little')
    assert(length > 0xa0)
    length -= 0xa0
    offset += 1
    v = bytes[offset:offset+length]
    print(f'offset = {hex(offset)}, bytes = {v}')
    offset += length
    return offset, v

def read_unknown_bytes(bytes, offset, length):
    v = bytes[offset:offset+length]
    print(f'offset = {hex(offset)}, unknown bytes = {v.hex()}')
    offset += length
    return offset, v



In [87]:
# try to parse sample_charcard_face_0_extra_bytes.4.bin
ex4_bytes = read_bytes('sample_charcard_face_0_extra_bytes.4.bin')
offset = 0

offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
print()

offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
print()

offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 3)
print()
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
print()

offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 3)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 3)
print()
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
print()

offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_a_bytes(ex4_bytes, offset)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 5)
print()
offset, _ = read_a_bytes(ex4_bytes, offset)
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
offset, _ = read_unknown_bytes(ex4_bytes, offset, 1)
print()
print()


offset = 0x0, unknown bytes = 81


offset = 0x2, bytes = b'lstInfo'
offset = 0x9, unknown bytes = 98

offset = 0xa, unknown bytes = 84


offset = 0xc, bytes = b'name'
offset = 0x11, bytes = b'Custom'

offset = 0x18, bytes = b'version'
offset = 0x20, bytes = b'0.0.0'

offset = 0x26, bytes = b'pos'
offset = 0x29, unknown bytes = 00

offset = 0x2b, bytes = b'size'
offset = 0x2f, unknown bytes = cd0c64

offset = 0x32, unknown bytes = 84


offset = 0x34, bytes = b'name'
offset = 0x39, bytes = b'Coordinate'

offset = 0x44, bytes = b'version'
offset = 0x4c, bytes = b'0.0.0'

offset = 0x52, bytes = b'pos'
offset = 0x55, unknown bytes = cd0c64

offset = 0x59, bytes = b'size'
offset = 0x5d, unknown bytes = cdfbc8

offset = 0x60, unknown bytes = 84


offset = 0x62, bytes = b'name'
offset = 0x67, bytes = b'Parameter'

offset = 0x71, bytes = b'version'
offset = 0x79, bytes = b'0.0.0'

offset = 0x7f, bytes = b'pos'
offset = 0x82, unknown bytes = ce0001082c

offset = 0x88, bytes = b'size'
offset = 0x

In [88]:
# new func to parse sample_charcard_face_0_extra_bytes.4.bin

def read_x_bytes(bytes, offset, exc={}):
    offset0 = offset

    if offset in exc:
        v = bytes[offset:offset+exc[offset]]
        offset += exc[offset]
        print(f'EXCEPTION: offset = {hex(offset0)}, v = {v.hex()}')
        return offset, v

    b0 = bytes[offset:offset+1]
    i0 = int.from_bytes(b0, 'big')
    offset += 1

    if i0 < 0xa0:
        v = b0
        print(f'offset = {hex(offset0)}, b0 = {b0.hex()}')
        return offset, v

    elif i0 < 0xc0:
        llen = i0 - 0xa0
        v = bytes[offset:offset+llen]
        offset += llen
        print(f'offset = {hex(offset0)}, v = {v}')
        return offset, v

    elif i0 == 0xca:
        llen = 4
        v = bytes[offset:offset+llen]
        offset += llen
        print(f'offset = {hex(offset0)}, v = {v.hex()}')
        return offset, v

    elif i0 == 0xc2:
        v = b0
        print(f'offset = {hex(offset0)}, b0 = {b0.hex()}')
        return offset, v

    elif i0 == 0xcd:
        llen = 2
        v = bytes[offset:offset+llen]
        offset += llen
        print(f'offset = {hex(offset0)}, v = {v.hex()}')
        return offset, v

    elif i0 == 0xce:
        llen = 4
        v = bytes[offset:offset+llen]
        offset += llen
        print(f'offset = {hex(offset0)}, v = {v.hex()}')
        return offset, v

    elif i0 == 0xdc:
        llen = 2
        v = bytes[offset:offset+llen]
        offset += llen
        print(f'offset = {hex(offset0)}, v = {v.hex()}')
        return offset, v

    else:
        raise Exception(f'invalid b0: {b0.hex()}, offset = {hex(offset0)}')


In [89]:
# parse sample_charcard_face_0_extra_bytes.4.bin using read_x_bytes

ex4_bytes = read_bytes('sample_charcard_face_0_extra_bytes.4.bin')
offset = 0

while offset < len(ex4_bytes):
    offset, _ = read_x_bytes(ex4_bytes, offset)


offset = 0x0, b0 = 81
offset = 0x1, v = b'lstInfo'
offset = 0x9, b0 = 98
offset = 0xa, b0 = 84
offset = 0xb, v = b'name'
offset = 0x10, v = b'Custom'
offset = 0x17, v = b'version'
offset = 0x1f, v = b'0.0.0'
offset = 0x25, v = b'pos'
offset = 0x29, b0 = 00
offset = 0x2a, v = b'size'
offset = 0x2f, v = 0c64
offset = 0x32, b0 = 84
offset = 0x33, v = b'name'
offset = 0x38, v = b'Coordinate'
offset = 0x43, v = b'version'
offset = 0x4b, v = b'0.0.0'
offset = 0x51, v = b'pos'
offset = 0x55, v = 0c64
offset = 0x58, v = b'size'
offset = 0x5d, v = fbc8
offset = 0x60, b0 = 84
offset = 0x61, v = b'name'
offset = 0x66, v = b'Parameter'
offset = 0x70, v = b'version'
offset = 0x78, v = b'0.0.0'
offset = 0x7e, v = b'pos'
offset = 0x82, v = 0001082c
offset = 0x87, v = b'size'
offset = 0x8c, b0 = 7a
offset = 0x8d, b0 = 84
offset = 0x8e, v = b'name'
offset = 0x93, v = b'Status'
offset = 0x9a, v = b'version'
offset = 0xa2, v = b'0.0.0'
offset = 0xa8, v = b'pos'
offset = 0xac, v = 000108a6
offset = 0xb1, 

In [90]:
# parse sample_charcard_face_0_extra_bytes.5.bin using read_x_bytes

ex4_bytes = read_bytes('sample_charcard_face_0_extra_bytes.5.bin')
offset = 0x0b # skip the first 11 bytes

exc = {
    0x1f9: 8,
    0x498: 3,
    0x96e: 7,
    0xc68: 9,
}

while offset < len(ex4_bytes):
    offset, _ = read_x_bytes(ex4_bytes, offset, exc)


offset = 0xb, v = b'version'
offset = 0x13, v = b'0.0.2'
offset = 0x19, v = b'shapeValueFace'
offset = 0x28, v = 0033
offset = 0x2b, v = 00000000
offset = 0x30, v = 3d75c28f
offset = 0x35, v = 00000000
offset = 0x3a, v = 3ed70a3d
offset = 0x3f, v = 3eb0a3d7
offset = 0x44, v = 3f0f5c29
offset = 0x49, v = 3f4ccccd
offset = 0x4e, v = 3f451eb8
offset = 0x53, v = 3e3d70a4
offset = 0x58, v = 3f800000
offset = 0x5d, v = 3ebd70a4
offset = 0x62, v = 00000000
offset = 0x67, v = 00000000
offset = 0x6c, v = 3f800000
offset = 0x71, v = 3f800000
offset = 0x76, v = 3f020d49
offset = 0x7b, v = 3f28f5c3
offset = 0x80, v = 3f2147ae
offset = 0x85, v = 3ef5c28f
offset = 0x8a, v = 3eca3d71
offset = 0x8f, v = 3ee8f5c3
offset = 0x94, v = 3f147ae1
offset = 0x99, v = 3f028f5c
offset = 0x9e, v = 3edeb852
offset = 0xa3, v = 3edeb852
offset = 0xa8, v = 3f800000
offset = 0xad, v = 3f800000
offset = 0xb2, v = 3f3d70a4
offset = 0xb7, v = 3f07ae15
offset = 0xbc, v = 3f0ccccd
offset = 0xc1, v = 3ef0a3d6
offset = 0xc6,

Exception: invalid b0: f2, offset = 0x513e